In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns

from scipy.stats import pearsonr

sns.set_style("darkgrid")
np.random.seed(930525)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)

warnings.simplefilter('once')

%matplotlib inline
%load_ext watermark
%watermark --iversions

seaborn 0.10.1
pandas  1.0.4
numpy   1.18.4



In [4]:
from collections import defaultdict

url = requests.get('https://docs.google.com/spreadsheets/d/1xqhFLsQplNUu5k9Uxv7DgTwftCiE78E0JmsKvNi0Ft4/export?format=csv')
csv_raw = StringIO(url.text)
df_truth = pd.read_csv(csv_raw)

inf_tax_file = "/mnt/btrfs/data/rep202/ab/taxmap.tsv"

In [5]:
dd = defaultdict(set)
dd_species = defaultdict(set)
for t in df_truth.itertuples():
    query = t.homotypic_synonym
    results = !rg {query} {inf_tax_file}
    if len(results) > 0:
        for r in results:
            split = r.split("\t")
            dd[t.dataset].add(split[0])
            dd_species[t.dataset].add(";".join(split[1].split(";")[:7]))

In [4]:
from glob import glob
import os

basepath = "/mnt/btrfs/data/type_1/species_mc"
dirs = [d for d in os.listdir(basepath) if os.path.isdir(os.path.join(basepath, d))]

# dual_index failed
dirs.remove('dual_index')

In [22]:
import random

basepath_sim = "/mnt/btrfs/data/type_1/simulation_strain"

os.makedirs(basepath_sim, exist_ok=True)
os.makedirs(basepath_sim + "/strain_hits", exist_ok=True)
os.makedirs(basepath_sim + "/sims", exist_ok=True)


for directory in dirs:
    strains = list(dd[directory])
    strain = strains[random.randint(0, len(strains) - 1)]
    
    alignment_file = os.path.join(basepath, directory, 'filter', 'allpath.r202.b6')
    !grep {strain} {alignment_file} > {basepath_sim}/strain_hits/{strain}.{directory}.b6
    
    with open(f"{basepath_sim}/strain_hits/{strain}.{directory}.b6", "r") as inf:
        lines = inf.readlines()
    
    for i in [10, 100, 1000, 10_000, 100_000, 1_000_000]:
        for j in range(10):
            with open(f"{basepath_sim}/sims/{strain}.{directory}.{j}.{i}.b6", "w") as outf:
                outf.writelines(random.choices(lines, k=i))

In [23]:
import random

basepath_sim = "/mnt/btrfs/data/type_1/simulation_metagenome"

os.makedirs(basepath_sim, exist_ok=True)

for directory in dirs:
    strains = list(dd[directory])
    strain = strains[random.randint(0, len(strains) - 1)]
    
    alignment_file = os.path.join(basepath, directory, 'filter', 'allpath.r202.b6')
    
    with open(alignment_file, "r") as inf:
        lines = inf.readlines()
    
    for i in [10, 100, 1000, 10_000, 100_000, 1_000_000]:
        for j in range(10):
            with open(f"{basepath_sim}/{directory}.{j}.{i}.b6", "w") as outf:
                outf.writelines(random.choices(lines, k=i))

In [31]:
basepath_sim = "/mnt/btrfs/data/type_1/simulation_strain/sims"
db_features = "/mnt/nvidia/pkr/code/type_1/data/db_features.csv"


from glob import glob
b6_files = glob(f"{basepath_sim}/*.b6", recursive=True)

for b6_file in b6_files:
    base_path = os.path.dirname(b6_file)
    basename = os.path.basename(".".join(b6_file.split(".")[:-1]))

    command_coverage = f"""/usr/bin/time -v python /mnt/nvidia/pkr/code/type_1/type_1/main.py features-alignment \
{db_features} \
{b6_file} \
{basepath_sim}/t1.coverage.{basename}.txt \
> {basepath_sim}/t1.coverage.{basename}.log 2>&1"""
    !{command_coverage}

In [ ]:
basepath_sim = "/mnt/btrfs/data/type_1/simulation_metagenome"
db_features = "/mnt/nvidia/pkr/code/type_1/data/db_features.csv"


from glob import glob
b6_files = glob(f"{basepath_sim}/*.b6", recursive=True)

for b6_file in b6_files:
    base_path = os.path.dirname(b6_file)
    basename = os.path.basename(".".join(b6_file.split(".")[:-1]))

    command_coverage = f"""/usr/bin/time -v python /mnt/nvidia/pkr/code/type_1/type_1/main.py features-alignment \
{db_features} \
{b6_file} \
{basepath_sim}/t1.coverage.{basename}.txt \
> {basepath_sim}/t1.coverage.{basename}.log 2>&1"""
    !{command_coverage}

In [ ]:
basepath_sim = "/mnt/btrfs/data/type_1/simulation_metagenome"
db_features = "/mnt/nvidia/pkr/code/type_1/data/db_features.csv"


from glob import glob
b6_files = glob(f"{basepath_sim}/mbarc*.b6", recursive=True)

for b6_file in b6_files:
    base_path = os.path.dirname(b6_file)
    basename = os.path.basename(".".join(b6_file.split(".")[:-1]))

    command_coverage = f"""/usr/bin/time -v python /mnt/nvidia/pkr/code/type_1/type_1/main.py features-alignment \
{db_features} \
{b6_file} \
{basepath_sim}/t1.coverage.{basename}.txt \
> {basepath_sim}/t1.coverage.{basename}.log 2>&1"""
    !{command_coverage}